# 基于textrank的实体抽取
### 1.爬取网上一篇文章并保存

In [1]:
import requests
from lxml import html

etree=html.etree
url='http://news.cri.cn/gb/27824/2013/01/28/6611s4005015.htm'
data=requests.get(url)
#data.encoding=''
#print(data)
s=etree.HTML(data.text)
text1=s.xpath('//div[@id="ccontent"]/p/text()')#得到的文本是一个列表，里面有6项，代表6个自然段
title=s.xpath('//div[@class="news_ts_tit"]/text()')[0].strip()#[0]是取标题的第一项，trip()去掉首尾空格
print("爬取文本：\n","标题：",title,"\n正文：",text1)
text=title

# 将得到的文本写入文件
for i in range(0,len(text1)-1):
   text += text1[i]
sentence_list=[]
print(text)
title='dat/' + title + '.txt'
with open(title, 'w', encoding='utf-8') as f:
   f.writelines(text)

爬取文本：
 标题： 吴邦国:中国坚持和平发展不会因国力地位变化而改变 
正文： ['\u3000\u3000国际在线消息（记者 吴倩）：亚太地区影响最大的议会间组织――亚太议会论坛28日召在符拉迪沃斯托克召开年会。中国全国人大常委会委员长吴邦国在与会发言中重申，坚持和平发展道路是中国基于时代发展潮流和自身根本利益作出的战略抉择，不会因为综合国力和国际地位的变化而改变。 \r\n', '\u3000\u3000亚太议会论坛成立于1993年，拥有中国、俄罗斯、美国等27个成员国。年会是这个论坛的最高决策机构，每年轮流在太平洋两岸举行。据了解，本届年会为期3天，与会代表将围绕地区安全、经济贸易、区域合作等议题进行坦诚对话。 \r\n', '\u3000\u3000吴邦国在年会主旨发言中阐述了中方在事关亚太地区和平发展重大问题上的原则主张。他指出，促进亚太地区和平合作发展，是亚太各国的共同责任，各方要从战略高度审视地区形势和彼此关系，努力扩大共识、付诸行动。“我们要摒弃冷战思维和零和博弈观念，相互尊重彼此主权和核心利益，推动建立公平有效的地区安全机制；要积极推动高新技术、先进制造、节能环保、能源资源、现代农业等领域务实合作，反对各种形式的保护主义，推动贸易和投资自由化、区域经济一体化；要尊重文明多样性，尊重各国人民自主选择的发展道路，促进不同文明和社会制度相互交流借鉴，推动亚太多元文明共同进步。” \r\n', '\u3000\u3000吴邦国表示，议会在各自国家政治生活中发挥着重要作用。各成员的议会应敦促和支持本国政府实施有利于和平发展、互利共赢的外交政策，加强议会间各层次、各领域对话交流，努力使议会交往成为推动国家关系发展的建设性力量。他同时强调，中国坚持和平发展，继续推进对外开放。“中国已经取得的发展成就举世瞩目，但面临的矛盾和挑战也世所罕见。发展仍然是解决中国所有问题的关键，我们将一心一意谋发展，始终不渝奉行互利共赢开放战略，努力在更广领域、更高层次扩大对外开放。我们将始终不渝走和平发展道路，坚定奉行独立自主的和平外交政策，这是中国基于时代发展潮流和自身根本利益作出的战略抉择，不会因为中国综合国力和国际地位的变化而改变。” \r\n', '\u3000\u3000吴邦国表示，中国坚持国家不论大小、强弱、贫富一律平等，不干涉别国内政，永不称霸；坚持通过和平谈判方式解

### 2.打开文件

In [2]:
import numpy as np
import re,jieba.posseg as pseg
from itertools import chain

#打开文件
sentences_list = []
file_path='dat/吴邦国:中国坚持和平发展不会因国力地位变化而改变.txt'
fp = open(file_path,'r',encoding="utf8")
for line in fp.readlines():
        if line.strip():
            # 把元素按照[。！；？]进行分隔，得到句子。
            line_split = re.split(r'[。！；？]',line.strip())
            # [。！；？]这些符号也会划分出来，把它们去掉。
            line_split = [line.strip() for line in line_split if line.strip() not in ['。','！','？','；'] and len(line.strip())>1]
            sentences_list.append(line_split)
sentences_list = list(chain.from_iterable(sentences_list))
print("前10个句子为：\n")
print(sentences_list[:10])
print("句子总数：", len(sentences_list))

前10个句子为：

['吴邦国:中国坚持和平发展不会因国力地位变化而改变\u3000\u3000国际在线消息（记者 吴倩）：亚太地区影响最大的议会间组织――亚太议会论坛28日召在符拉迪沃斯托克召开年会', '中国全国人大常委会委员长吴邦国在与会发言中重申，坚持和平发展道路是中国基于时代发展潮流和自身根本利益作出的战略抉择，不会因为综合国力和国际地位的变化而改变', '亚太议会论坛成立于1993年，拥有中国、俄罗斯、美国等27个成员国', '年会是这个论坛的最高决策机构，每年轮流在太平洋两岸举行', '据了解，本届年会为期3天，与会代表将围绕地区安全、经济贸易、区域合作等议题进行坦诚对话', '吴邦国在年会主旨发言中阐述了中方在事关亚太地区和平发展重大问题上的原则主张', '他指出，促进亚太地区和平合作发展，是亚太各国的共同责任，各方要从战略高度审视地区形势和彼此关系，努力扩大共识、付诸行动', '“我们要摒弃冷战思维和零和博弈观念，相互尊重彼此主权和核心利益，推动建立公平有效的地区安全机制', '要积极推动高新技术、先进制造、节能环保、能源资源、现代农业等领域务实合作，反对各种形式的保护主义，推动贸易和投资自由化、区域经济一体化', '要尊重文明多样性，尊重各国人民自主选择的发展道路，促进不同文明和社会制度相互交流借鉴，推动亚太多元文明共同进步']
句子总数： 19


### 3.分词，停用词

In [3]:
#加载停用词
stoplist= [word.strip() for word in open('stopwords.txt',encoding='utf-8').readlines()]
# print(stoplist)
word_flag = {}

# 对句子进行分词
def seg_depart(sentence):
    # 去掉非汉字字符
    sentence = re.sub(r'[^\u4e00-\u9fa5]+','',sentence)
    sentence_depart = pseg.cut(sentence.strip())
    word_list = []
    for word,flag in sentence_depart:
        if word not in stoplist:
            word_list.append(word)
            word_flag[word] = flag
    # 如果句子整个被过滤掉了，如：'02-2717:56'被过滤，那就返回[],保持句子的数量不变
    return word_list

sentence_word_list = []
for sentence in sentences_list:
    line_seg = seg_depart(sentence)
    sentence_word_list.append(line_seg)
print("一共有",len(sentences_list),'个句子。\n')
print("前10个句子分词后的结果为：\n",sentence_word_list[:10])

# 保证处理后句子的数量不变，我们后面才好根据textrank值取出未处理之前的句子作为摘要。
if len(sentences_list) == len(sentence_word_list):
    print("\n数据预处理后句子的数量不变！")

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/19/dgxwsbgd76728hx577833nx00000gn/T/jieba.cache
Loading model cost 0.661 seconds.
Prefix dict has been built successfully.


一共有 19 个句子。

前10个句子分词后的结果为：
 [['吴邦国', '中国', '坚持', '和平', '发展', '不会', '国力', '地位', '变化', '改变', '国际', '在线', '消息', '记者', '吴倩', '亚太地区', '影响', '最大', '议会', '间', '组织', '亚太', '议会', '论坛', '日', '召在', '符拉迪沃斯托克', '召开', '年', '会'], ['中国', '全国人大常委会', '委员长', '吴邦国', '与会', '发言', '中', '重申', '坚持', '和平', '发展', '道路', '中国', '基于', '时代', '发展', '潮流', '根本利益', '作出', '战略', '抉择', '不会', '综合国力', '国际', '地位', '变化', '改变'], ['亚太', '议会', '论坛', '成立', '年', '拥有', '中国', '俄罗斯', '美国', '成员国'], ['年', '会', '论坛', '最高', '决策机构', '每年', '轮流', '太平洋', '两岸', '举行'], ['了解', '本届', '年', '会', '为期', '天', '与会代表', '围绕', '地区', '安全', '经济', '贸易', '区域合作', '议题', '进行', '坦诚', '对话'], ['吴邦国', '年', '会', '主旨', '发言', '中', '阐述', '中方', '事关', '亚太地区', '和平', '发展', '重大', '问题', '上', '原则', '主张'], ['指出', '促进', '亚太地区', '和平', '合作', '发展', '亚太', '各国', '共同', '责任', '各方', '战略', '高度', '审视', '地区', '形势', '关系', '努力', '扩大', '共识', '付诸行动'], ['摒弃', '冷战', '思维', '零', '博弈', '观念', '相互尊重', '主权', '核心', '利益', '推动', '建立', '公平', '有效', '地区', '安全', '机制'], ['积极', '推动', '高新技术', '先进', '制造', '节能', 

### 4.词汇

In [4]:
#fit_on_texts函数可以将输入的文本中的每个词编号，
#编号是根据词频的，词频越大，编号越小
from keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer()
tokenizer.fit_on_texts(sentence_word_list)
vocab = tokenizer.word_index  # 得到每个词的编号

print(vocab)

{'发展': 1, '中国': 2, '和平': 3, '议会': 4, '推动': 5, '吴邦国': 6, '坚持': 7, '年': 8, '亚太': 9, '会': 10, '战略': 11, '国家': 12, '不会': 13, '地位': 14, '变化': 15, '改变': 16, '国际': 17, '亚太地区': 18, '论坛': 19, '中': 20, '道路': 21, '地区': 22, '努力': 23, '领域': 24, '文明': 25, '间': 26, '发言': 27, '基于': 28, '时代': 29, '潮流': 30, '根本利益': 31, '作出': 32, '抉择': 33, '综合国力': 34, '安全': 35, '经济': 36, '贸易': 37, '对话': 38, '问题': 39, '促进': 40, '合作': 41, '各国': 42, '关系': 43, '扩大': 44, '主权': 45, '尊重': 46, '交流': 47, '表示': 48, '发挥': 49, '作用': 50, '互利': 51, '共': 52, '赢': 53, '外交政策': 54, '对外开放': 55, '解决': 56, '始终不渝': 57, '奉行': 58, '更': 59, '国力': 60, '在线': 61, '消息': 62, '记者': 63, '吴倩': 64, '影响': 65, '最大': 66, '组织': 67, '日': 68, '召在': 69, '符拉迪沃斯托克': 70, '召开': 71, '全国人大常委会': 72, '委员长': 73, '与会': 74, '重申': 75, '成立': 76, '拥有': 77, '俄罗斯': 78, '美国': 79, '成员国': 80, '最高': 81, '决策机构': 82, '每年': 83, '轮流': 84, '太平洋': 85, '两岸': 86, '举行': 87, '了解': 88, '本届': 89, '为期': 90, '天': 91, '与会代表': 92, '围绕': 93, '区域合作': 94, '议题': 95, '进行': 96, '坦诚': 97, '主旨': 98, '阐述'

### 5.利用FastText得到Embedding

In [5]:
from gensim.models.fasttext import FastText
import time

class FTModel:
    def __init__(self, model_path):
        print('FastText word2vec model loading...', end='')
        start = time.time()
        self.model = FastText.load_fasttext_format(model_path)
        end = time.time()
        print('model loaded successfully, elapsed: %.2f s' % (end - start))

In [7]:
ft_model = FTModel('../../../../../models/FastText/cc.zh.300.bin.gz')

word_embeddings = {}
for word, i in vocab.items():
    try:
        # model.wv[word]存的就是这个word的词向量
        word_embeddings[word] =ft_model.model.wv[word]
    except KeyError:
        continue
print('词汇量：', len(vocab))
print('输出了：', len(word_embeddings),'个词')

FastText word2vec model loading...

/var/folders/19/dgxwsbgd76728hx577833nx00000gn/T/ipykernel_4761/352868489.py:8: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  self.model = FastText.load_fasttext_format(model_path)


model loaded successfully, elapsed: 254.34 s
词汇量： 222
输出了： 222 个词


### 6.词语embedding的平均值作为句子的向量表示

In [7]:
#求句子最大长度
# maxLen=0
# for sentence in sentences_list:
#     length=0
#     for wd in sentence:
#         length=length+1
#     if (length>maxLen):maxLen=length

sentence_vectors = []
for line in sentence_word_list:
    if len(line)!=0:
        # 如果句子中的词语不在字典中，那就把embedding设为300维元素为0的向量。
        # 得到句子中全部词的词向量后，求平均值，得到句子的向量表示
        #TypeError: type numpy.ndarray doesn't define __round__ method,将round改为np.round
        v = np.round(sum(word_embeddings.get(word, np.zeros((300,))) for word in line)/(len(line)))
    else:
        # 如果句子为[]，那么就向量表示为300维元素为0个向量。
        v = np.zeros((300,))
    sentence_vectors.append(v)

### 7.摘要生成

In [21]:
#计算句子之间的余弦相似度，构成相似度矩阵
sim_mat = np.zeros([len(sentences_list), len(sentences_list)])

from sklearn.metrics.pairwise import cosine_similarity

for i in range(len(sentences_list)):
  for j in range(len(sentences_list)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,300), sentence_vectors[j].reshape(1,300))[0,0]
print("句子相似度矩阵的形状为：",sim_mat.shape)

#迭代得到句子的textrank值，排序并取出摘要"""
import networkx as nx

# 利用句子相似度矩阵构建图结构，句子为节点，句子相似度为转移概率
nx_graph = nx.from_numpy_array(sim_mat)

# 得到所有句子的textrank值
scores = nx.pagerank(nx_graph)

# 根据textrank值对未处理的句子进行排序
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences_list)), reverse=True)

# 取出得分最高的前3个句子作为摘要
sn = 3
for i in range(sn):
    print("第"+str(i+1)+"条摘要：\n\n",ranked_sentences[i][1],'\n')


句子相似度矩阵的形状为： (19, 19)
第1条摘要：

 要积极推动高新技术、先进制造、节能环保、能源资源、现代农业等领域务实合作，反对各种形式的保护主义，推动贸易和投资自由化、区域经济一体化 

第2条摘要：

 要尊重文明多样性，尊重各国人民自主选择的发展道路，促进不同文明和社会制度相互交流借鉴，推动亚太多元文明共同进步 

第3条摘要：

 推动和平解决国际争端和热点问题，发挥负责任大国作用 

